In [6]:
import os
from pathlib import Path
import pandas as pd
from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from dotenv import load_dotenv
load_dotenv()

True

## Using Tesla to create Vader sentiment

In [7]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [8]:
# Reading the News API key enviroment variable
api_key = os.getenv("news_api")


In [9]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)


In [10]:
# Fetch all the news about inflation
inflation_headlines = newsapi.get_everything(
    q="inflation",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

# Print total articles
print(f"Total articles about inflation: {inflation_headlines['totalResults']}")

# Show sample article
inflation_headlines["articles"][0]

Total articles about inflation: 14645


{'source': {'id': None, 'name': 'New York Times'},
 'author': 'David Leonhardt',
 'title': 'The Specter of Inflation',
 'description': 'Is it haunting the U.S. economy?',
 'url': 'https://www.nytimes.com/2021/06/17/briefing/inflation-us-economy-covid-pandemic.html',
 'urlToImage': 'https://static01.nyt.com/images/2021/06/17/lens/17ambriefing-promo/17ambriefing-inflation-facebookJumbo-v2.jpg',
 'publishedAt': '2021-06-17T10:30:22Z',
 'content': 'With excess saving they can afford more of everything, Jason Furman, a Harvard economist and former Obama administration official, wrote this week.\r\nAlthough companies are increasing the supply of ma… [+2099 chars]'}

In [12]:
# Create the inflation sentiment scores DataFrame
inflation_sentiments = []

for article in inflation_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        inflation_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
inflation_df = pd.DataFrame(inflation_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
inflation_df = inflation_df[cols]

inflation_df.head()


,date,text,compound,positive,negative,neutral
0,2021-06-17,With excess saving they can afford more of eve...,0.0000,0.00,0.000,1.000
1,2021-06-21,"10. And finally, figuring out how to be a supe...",0.0000,0.00,0.000,1.000
2,2021-06-10,"4. As the U.S. withdraws from Afghanistan, int...",-0.4939,0.00,0.091,0.909
3,2021-06-16,"A day after ousting two top executives, the el...",0.2023,0.05,0.000,0.950
4,2021-06-10,,0.0000,0.00,0.000,0.000


In [13]:
inflation_df.describe()


,compound,positive,negative,neutral
count,100.000000,100.000000,100.000000,100.000000
mean,0.123166,0.053600,0.026420,0.909940
std,0.331568,0.065226,0.049701,0.125199
min,-0.835000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.854250
50%,0.000000,0.015500,0.000000,0.936000
75%,0.340000,0.085000,0.060000,1.000000
max,0.840200,0.237000,0.245000,1.000000
